# When's my Green Card coming?

Finding any information on when your government documents are arriving is painful and relies knowing about government websites that are not geared towards automated alerts (API anyone?).

This notebook will help programatically extract processing dates for green cards.

First, we need to establish a session with the USCIS "eGov" website (https://egov.uscis.gov/cris/processTimesDisplayInit.do) and grab a session cookie.

**Disclaimer:** per https://www.uscis.gov/website-policies and https://www.uscis.gov/website-policies/privacy-and-legal-disclaimers there do not seem to be any restrictions on this sort of automated scripting. But be wise, and have fun!

In [192]:
import requests
url="https://egov.uscis.gov/cris/processTimesDisplayInit.do"
r = requests.get(url)
sessionId=r.headers['Set-Cookie'].split(';')[0].split('=')[1]
print sessionId

abc4A-De1h0x6dgqxNH6v


Next, we can query the page for the processing center of interest. To find your Service Center, check your notification document and crosscheck with the service centers listed at https://egov.uscis.gov/cris/processTimesDisplayInit.do (extracting the numerical value). At the time of writing:

|code|        Service Center         |
|----|-------------------------------|
|991 |CSC - California Service Center|
|992 |NSC - Nebraska Service Center  |
|993 |TSC - Texas Service Center     |
|990 |VSC - Vermont Service Center   |
|1031|YSC - Potomac Service Center   |

If you'd like to check other processing dates, check the name and value of the submit value and replace `'displaySCProcTimes':'Service Center Processing Dates'` accordingly.

In [193]:
serviceCenter='992' # Nebraska
url="https://egov.uscis.gov/cris/processingTimesDisplay.do"
cookies = dict(JSESSIONID=sessionId)
payload = {'serviceCenter': serviceCenter, 'displaySCProcTimes':'Service Center Processing Dates'}
r = requests.post(url, data=payload, cookies=cookies)
if r.status_code == 200:
    print "Request successful!"
else:
    print "Something went wrong with the request", r.status_code

Request successful!


Now we need to parse the response data in `r.text`. This is where things get a little messy. We're going to use Beautiful Soup:

In [194]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text,'html.parser')
print soup.prettify()

<!DOCTYPE html PUBLIC "-//W3C/DTD HTML 4.01//EN"  "http://www.w3.org/TR/html4/DTD/strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   U.S. Citizenship and Immigration Services - 
      
      
      
      USCIS Processing Time Information
  </title>
  <link href="https://my.uscis.gov/assets/favicon-9429d1fce1555403bc766c9a06e15410.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/cris/style/branding/uscis.css" rel="stylesheet"/>
  <link href="/cris/style/cris.css" rel="stylesheet"/>
  <!--[if lte IE 6]>
    <link rel="stylesheet" href="/cris/style/ie6.css" />
    <![endif]-->
  <link href="/cris/style/processingTimes.css" rel="stylesheet"/>
  <script src="/cris/scripts/cris_library.js" type="text/javascript">
  </script>
  <script src="/cris/scripts/yui/yahoo-min.js" type="text/javascript">
  </script>
  <script src="/cris/scripts/yui/event-min.js" type=

First, we need to extract when they last updated the data, so we can estimate actual green card status:

In [228]:
from datetime import datetime
lastUpdated = soup.caption.text.split(':')[1].strip()
lastUpdatedDateTime = datetime.strptime(lastUpdated,'%B %d, %Y')
print lastUpdated

July 31, 2017


Now, we need to search for the specific visa class of interest, and extract it. This scraping may differ depending on your visa class. Here we are looking for a form I-140:

In [197]:
formToFind = 'I-140'
forms=soup.find_all('tbody')
for form in forms:
    if form.attrs['title'] == formToFind:
        break

print form

<tbody class="even" title="I-140">
<tr class="first">
<th scope="row">I-140</th>
<td>Immigrant Petition for Alien Worker</td>
<td>Multinational executive or manager  (E13)</td>
<td>
				
				
				
				
				July 2, 2016
				
				</td>
</tr>
<tr class="">
<th scope="row">I-140</th>
<td>Immigrant Petition for Alien Worker</td>
<td>Advanced degree or exceptional ability requesting a National Interest Waiver (NIW)</td>
<td>
				
				
				
				
				August 2, 2016
				
				</td>
</tr>
<tr class="">
<th scope="row">I-140</th>
<td>Immigrant Petition for Alien Worker</td>
<td>Schedule A Nurses</td>
<td>
				
				
				
				
				November 2, 2016
				
				</td>
</tr>
<tr class="">
<th scope="row">I-140</th>
<td>Immigrant Petition for Alien Worker</td>
<td>Extraordinary ability (E11)</td>
<td>
				
				
				
				
				March 1, 2017
				
				</td>
</tr>
<tr class="">
<th scope="row">I-140</th>
<td>Immigrant Petition for Alien Worker</td>
<td>Outstanding professor or researcher (E12)</td>
<td>
				
			

Within this group, we need to pull out (E13) classification processing date:

In [226]:
classification='E13'
for tr in form.find_all('tr'):
    for td in tr.find_all('td'):
        if td.text.find("(%s)"%classification) != -1:
            processingDate = td.next_sibling.next_sibling.text.strip()
            break
processingDateTime = datetime.strptime(processingDate,'%B %d, %Y')
print processingDateTime.strftime("%B %d, %Y")

July 02, 2016


Great! So we now have the processing date of our green card, as per the last update of the eGov portal.

Given that the eGov portal isn't updated all that often, we're going to estimate a linear projection of the date when it was updated to today, to get us a little closer to our real date:

In [227]:
estimatedProcessingDateTime = datetime.now() - lastUpdatedDateTime + processingDateTime
print estimatedProcessingDateTime.strftime("%B %d, %Y")

August 22, 2016


That's it! You can now crosscheck this date with your notification date and figure out whether you should have heard something, or if you need to sit tight.

Enjoy!